In [ ]:
import pandas as pd
import string, re


!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [ ]:
pth = '/content/drive/MyDrive/Celerates School/06. Classification 3/'

In [ ]:
df = pd.read_csv(pth+'20191002-reviews.csv')
print(df.shape)
df.head()

(203787, 15)


,itemId,category,name,rating,originalRating,reviewTitle,reviewContent,likeCount,upVotes,downVotes,helpful,relevanceScore,boughtDate,clientType,retrievedDate
0,100002528,beli-harddisk-eksternal,Kamal U.,5,NaN,NaN,bagus mantap dah sesui pesanan,0,0,0,True,26.51,09 Apr 2019,androidApp,2019-10-02
1,100002528,beli-harddisk-eksternal,yofanca m.,4,NaN,NaN,"Bagus, sesuai foto",0,0,0,True,22.49,24 Sep 2017,androidApp,2019-10-02
2,100002528,beli-harddisk-eksternal,Lazada Customer,5,NaN,ok mantaaapppp barang sesuai pesanan.. good,okkkkk mantaaaaaaapppp ... goood,0,0,0,True,21.50,04 Apr 2018,androidApp,2019-10-02
3,100002528,beli-harddisk-eksternal,Lazada Customer,4,NaN,NaN,bagus sesuai,0,0,0,True,20.51,22 Sep 2017,androidApp,2019-10-02
4,100002528,beli-harddisk-eksternal,Yosep M.,5,NaN,NaN,NaN,0,0,0,True,16.01,17 Agu 2018,androidApp,2019-10-02


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203787 entries, 0 to 203786
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   itemId          203787 non-null  int64  
 1   category        203787 non-null  object 
 2   name            203787 non-null  object 
 3   rating          203787 non-null  int64  
 4   originalRating  8 non-null       float64
 5   reviewTitle     23399 non-null   object 
 6   reviewContent   107029 non-null  object 
 7   likeCount       203787 non-null  int64  
 8   upVotes         203787 non-null  int64  
 9   downVotes       203787 non-null  int64  
 10  helpful         203787 non-null  bool   
 11  relevanceScore  203787 non-null  float64
 12  boughtDate      196680 non-null  object 
 13  clientType      203787 non-null  object 
 14  retrievedDate   203787 non-null  object 
dtypes: bool(1), float64(2), int64(5), object(7)
memory usage: 22.0+ MB


In [ ]:
cols = ['rating','reviewContent']
df = df[cols]
df = df.dropna()
print(df.shape)
df.head()

(107029, 2)


,rating,reviewContent
0,5,bagus mantap dah sesui pesanan
1,4,"Bagus, sesuai foto"
2,5,okkkkk mantaaaaaaapppp ... goood
3,4,bagus sesuai
7,1,bima


In [ ]:
df.rating.value_counts()

rating
5    82896
4    10626
1     6951
3     4372
2     2184
Name: count, dtype: int64

In [ ]:
#mencari data dengan review lebih dari 1 kata
df = df[df.reviewContent.str.split().str.len()>1]
print(df.shape)
df.head()

(99280, 2)


,rating,reviewContent
0,5,bagus mantap dah sesui pesanan
1,4,"Bagus, sesuai foto"
2,5,okkkkk mantaaaaaaapppp ... goood
3,4,bagus sesuai
8,1,baru 10 bulan layarnya dah bergaris


In [ ]:
#seleksi data dengan rating 1,2,5
df = df[df.rating.isin([1,2,5])]
print(df.shape)
df.head()

(85051, 2)


,rating,reviewContent
0,5,bagus mantap dah sesui pesanan
2,5,okkkkk mantaaaaaaapppp ... goood
8,1,baru 10 bulan layarnya dah bergaris
9,5,"Pesan rabu sore,minggu sore sampe,,barang sesu..."
11,1,"Mau tanya ini cicilnya pake apa ya,cc bkn?"


In [ ]:
df['sentiment'] = ['positif' if x==5 else 'negatif' for x in df.rating]
print(df.shape)
df.head()

(85051, 3)


,rating,reviewContent,sentiment
0,5,bagus mantap dah sesui pesanan,positif
2,5,okkkkk mantaaaaaaapppp ... goood,positif
8,1,baru 10 bulan layarnya dah bergaris,negatif
9,5,"Pesan rabu sore,minggu sore sampe,,barang sesu...",positif
11,1,"Mau tanya ini cicilnya pake apa ya,cc bkn?",negatif


In [ ]:
df.sentiment.value_counts()

sentiment
positif    76090
negatif     8961
Name: count, dtype: int64

In [ ]:
#downsampling
# num_sample = len(df[df.sentiment=='negatif'])
num_sample = 600
ds_positif_label = df[df.sentiment=='positif'].sample(n=num_sample, random_state=1)
ds_positif_label.head()

,rating,reviewContent,sentiment
159699,5,mantap.. semoga awet barangnya.. maaf baru kas...,positif
71993,5,Seneng baranga cepat sampai packing juga rapih...,positif
35018,5,"Barang sampai dengan selamat,. berfungsi denga...",positif
68452,5,Brg bagus cukup memuaskan,positif
128179,5,Sesuai dengan pesanan,positif


In [ ]:
df = pd.concat([ds_positif_label,df[df.sentiment=='negatif'].iloc[:num_sample]],axis=0)
df.drop('rating',axis=1,inplace=True)
print(df.shape)
df.head()

(1200, 2)


,reviewContent,sentiment
159699,mantap.. semoga awet barangnya.. maaf baru kas...,positif
71993,Seneng baranga cepat sampai packing juga rapih...,positif
35018,"Barang sampai dengan selamat,. berfungsi denga...",positif
68452,Brg bagus cukup memuaskan,positif
128179,Sesuai dengan pesanan,positif


# Casefolding

In [ ]:
df['corpus'] = df.reviewContent.str.lower()
print(df.shape)
df.head()

(1200, 3)


,reviewContent,sentiment,corpus
159699,mantap.. semoga awet barangnya.. maaf baru kas...,positif,mantap.. semoga awet barangnya.. maaf baru kas...
71993,Seneng baranga cepat sampai packing juga rapih...,positif,seneng baranga cepat sampai packing juga rapih...
35018,"Barang sampai dengan selamat,. berfungsi denga...",positif,"barang sampai dengan selamat,. berfungsi denga..."
68452,Brg bagus cukup memuaskan,positif,brg bagus cukup memuaskan
128179,Sesuai dengan pesanan,positif,sesuai dengan pesanan


#Punctuation

In [ ]:
df['corpus'] = df.corpus.str.translate(str.maketrans('', '', string.punctuation))
print(df.shape)
df.head()

(1200, 3)


,reviewContent,sentiment,corpus
159699,mantap.. semoga awet barangnya.. maaf baru kas...,positif,mantap semoga awet barangnya maaf baru kasih p...
71993,Seneng baranga cepat sampai packing juga rapih...,positif,seneng baranga cepat sampai packing juga rapih...
35018,"Barang sampai dengan selamat,. berfungsi denga...",positif,barang sampai dengan selamat berfungsi dengan ...
68452,Brg bagus cukup memuaskan,positif,brg bagus cukup memuaskan
128179,Sesuai dengan pesanan,positif,sesuai dengan pesanan


# Slangword

In [ ]:
slang_dict = pd.read_csv(pth+'new_kamusalay.csv', encoding='latin1',names=['alay','normal'])
slang_dict = dict(zip(slang_dict.alay.tolist(),slang_dict.normal.tolist()))

In [ ]:
def cvt_slang(sentence):
  return ''.join(' ').join(t for t in [slang_dict[x] if x in slang_dict.keys() else x for x in sentence.split()])

In [ ]:
df['corpus'] = df.corpus.apply(cvt_slang)
print(df.shape)
df.head()

(1200, 3)


,reviewContent,sentiment,corpus
159699,mantap.. semoga awet barangnya.. maaf baru kas...,positif,mantap semoga awet barangnya maaf baru kasih p...
71993,Seneng baranga cepat sampai packing juga rapih...,positif,senang baranga cepat sampai packing juga rapi ...
35018,"Barang sampai dengan selamat,. berfungsi denga...",positif,barang sampai dengan selamat berfungsi dengan ...
68452,Brg bagus cukup memuaskan,positif,barang bagus cukup memuaskan
128179,Sesuai dengan pesanan,positif,sesuai dengan pesanan


#Repetition Character

In [ ]:
def repetition_char(txt):
  sub_str = re.findall(r"((\w)\2{2,})",txt)
  for x in sub_str:
    txt = txt.replace(x[0],x[1])
  return txt

In [ ]:
df['corpus'] = df.corpus.apply(repetition_char)
df = df.reset_index(drop=True)
print(df.shape)
df.head()

(1200, 3)


,reviewContent,sentiment,corpus
0,mantap.. semoga awet barangnya.. maaf baru kas...,positif,mantap semoga awet barangnya maaf baru kasih p...
1,Seneng baranga cepat sampai packing juga rapih...,positif,senang baranga cepat sampai packing juga rapi ...
2,"Barang sampai dengan selamat,. berfungsi denga...",positif,barang sampai dengan selamat berfungsi dengan ...
3,Brg bagus cukup memuaskan,positif,barang bagus cukup memuaskan
4,Sesuai dengan pesanan,positif,sesuai dengan pesanan


#Stemming

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['corpus'] = df.corpus.apply(lambda x : stemmer.stem(str(x)))
print(df.shape)
df.head()

(1200, 3)


,reviewContent,sentiment,corpus
0,mantap.. semoga awet barangnya.. maaf baru kas...,positif,mantap moga awet barang maaf baru kasih nilai
1,Seneng baranga cepat sampai packing juga rapih...,positif,senang baranga cepat sampai packing juga rapi ...
2,"Barang sampai dengan selamat,. berfungsi denga...",positif,barang sampai dengan selamat fungsi dengan baik
3,Brg bagus cukup memuaskan,positif,barang bagus cukup muas
4,Sesuai dengan pesanan,positif,sesuai dengan pesan


#Stopword

In [ ]:
stops = pd.read_csv(pth+'stopwordbahasa.csv',header=None)
stop_factory = StopWordRemoverFactory()
STOPS = stop_factory.get_stop_words()+stops[0].tolist()
STOPS = set(STOPS)

In [ ]:
df['corpus'] = df.corpus.apply(lambda c : ''.join(' ').join(x for x in c.split() if x not in STOPS))
print(df.shape)
df.head()

(1200, 3)


,reviewContent,sentiment,corpus
0,mantap.. semoga awet barangnya.. maaf baru kas...,positif,mantap moga awet barang maaf kasih nilai
1,Seneng baranga cepat sampai packing juga rapih...,positif,senang baranga cepat packing rapi terima kasih
2,"Barang sampai dengan selamat,. berfungsi denga...",positif,barang selamat fungsi
3,Brg bagus cukup memuaskan,positif,barang bagus muas
4,Sesuai dengan pesanan,positif,sesuai pesan


In [ ]:
#save to csv
df.to_csv(pth+'data_bersih.csv',index=False)